In [ ]:
import cv2
import numpy as np 

MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15

def alignImages(im1, im2):
    
    # Convert images to grayscale
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
    
    # Detect ORB features and compute descriptors
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)
    
    # Match features
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    matches = matcher.match(descriptors1, descriptors2, None)
    
    # Sort matches by score
    matches.sort(key = lambda x: x.distance, reverse = False)
    
    # Remove worst matches
    numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
    matches = matches[:numGoodMatches]
    
    # Draw top matches
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
    """
    cv2.imwrite("matches_123.jpg", imMatches)
    """
    
    # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype = np.float32)
    points2 = np.zeros((len(matches), 2), dtype = np.float32)
    
    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt
    
    # Find homography
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
    
    # Use homography
    height, width, channels = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))
    
    return im1Reg, h, imMatches

if __name__ == '__main__':
    
    # Read reference image
    refImage = "form1.jpg"
    imReference = cv2.imread(refImage, cv2.IMREAD_COLOR)
    
    # Read image to be aligned
    phImage = "123.jpg"
    im = cv2.imread(phImage, cv2.IMREAD_COLOR)
    
    # Align images 
    imReg, h, imMatches = alignImages(im, imReference)
   
    """
    # Write aligned image to disk. 
    outFilename = "123_aligned.jpg"
    print("Saving aligned image : ", outFilename)
    cv2.imwrite(outFilename, imReg)
    """
    # Print homography
    print("Homography : \n", h)
    
    # Display an original image, matches between the image and the form, + aligned image
    cv2.namedWindow('original', cv2.WINDOW_NORMAL)
    cv2.imshow('original',im)
    
    cv2.namedWindow('matches', cv2.WINDOW_NORMAL)
    cv2.imshow('matches',imMatches)
    
    cv2.namedWindow('aligned', cv2.WINDOW_NORMAL)
    cv2.imshow('aligned',imReg)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
   

 
    
  



        

Homography : 
 [[ 1.46454643e+00 -5.65054910e-01  4.41612529e+02]
 [ 2.36576005e-01  2.27544119e+00 -2.16827798e+03]
 [-1.47023280e-04  2.62857338e-04  1.00000000e+00]]
